<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-the-IBES-PERMNO-link-file" data-toc-modified-id="Load-the-IBES-PERMNO-link-file-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load the IBES PERMNO link file</a></span></li><li><span><a href="#Load-the-detail-IBES-history-file" data-toc-modified-id="Load-the-detail-IBES-history-file-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the detail IBES history file</a></span></li><li><span><a href="#Load-the-CRSP-file" data-toc-modified-id="Load-the-CRSP-file-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the CRSP file</a></span><ul class="toc-item"><li><span><a href="#Calculate-the-earning-surprises" data-toc-modified-id="Calculate-the-earning-surprises-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Calculate the earning surprises</a></span></li></ul></li><li><span><a href="#Load-Quarterly-Compustat" data-toc-modified-id="Load-Quarterly-Compustat-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load Quarterly Compustat</a></span></li><li><span><a href="#Calculate-earnings-surprises" data-toc-modified-id="Calculate-earnings-surprises-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculate earnings surprises</a></span><ul class="toc-item"><li><span><a href="#Get-the-stock-price-5-days-before-the-earnings-announcement" data-toc-modified-id="Get-the-stock-price-5-days-before-the-earnings-announcement-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Get the stock price 5 days before the earnings announcement</a></span></li></ul></li><li><span><a href="#Assign-S&amp;P-index" data-toc-modified-id="Assign-S&amp;P-index-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Assign S&amp;P index</a></span></li><li><span><a href="#Save-output" data-toc-modified-id="Save-output-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Save output</a></span></li></ul></div>

**NOTE** the procedures follow Livnat and Mendenhall (Journal of Accounting Research, 2006)

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from datetime import datetime
bday_us = CustomBusinessDay(calendar=USFederalHolidayCalendar())

# Load the IBES PERMNO link file

In [2]:
iclink = pd.read_csv('../../Data/IBES/ibes_ticker_permno_link_file.csv')
iclink = iclink[['TICKER', 'PERMNO', 'sdate', 'edate']]
iclink = iclink[iclink['PERMNO'].notnull()]
iclink['startdate'] = pd.to_datetime(iclink['sdate'], format='%d%b%Y')
iclink['enddate'] = pd.to_datetime(iclink['edate'], format='%d%b%Y')
iclink = iclink[['startdate', 'enddate', 'PERMNO', 'TICKER']]

# Load the detail IBES history file

In [5]:
ibes = pd.read_csv('../../Data/IBES/IBES_Detail_History_1970_2019.csv.gz')

In [6]:
ibes = ibes[['TICKER', 'CUSIP', 'OFTIC', 'CNAME', 'ANNDATS',
             'ESTIMATOR', 'ANALYS', 'PDF', 'VALUE',
             'FPEDATS', 'REVDATS',  'ANNDATS_ACT', 'ANNTIMS_ACT', 'ACTUAL']]

In [7]:
# remove observations with no actual reported earnings
ibes = ibes[ibes.ACTUAL.notnull()]

In [8]:
ibes['FPEDATS'] = pd.to_datetime(ibes['FPEDATS'])
ibes['REVDATS'] = pd.to_datetime(ibes['REVDATS'])
ibes['ANNDATS'] = pd.to_datetime(ibes['ANNDATS'])

# select the last estimate by broker-analyst
ibes = ibes.sort_values(by=['TICKER', 'FPEDATS', 'ESTIMATOR',
                            'ANALYS', 'ANNDATS', 'REVDATS'])
ibes = ibes.groupby(['TICKER', 'FPEDATS',
                     'ESTIMATOR', 'ANALYS']).last().reset_index()

In [9]:
# How many estimates are reported on primary and diluted basis?
ibes['p_count'] = np.where(ibes.PDF == 'P', 1, 0)
ibes['d_count'] = np.where(ibes.PDF == 'D', 1, 0)
p_count = ibes.groupby(['TICKER', 'FPEDATS'])['p_count'].sum().reset_index()
d_count = ibes.groupby(['TICKER', 'FPEDATS'])['d_count'].sum().reset_index()
ibes.drop(['p_count', 'd_count'], 1, inplace=True)
ibes = pd.merge(ibes, p_count, on=['TICKER', 'FPEDATS'], how='left')
ibes = pd.merge(ibes, d_count, on=['TICKER', 'FPEDATS'], how='left')

In [10]:
ibes = ibes[ibes['ANNDATS_ACT'].notnull()]
ibes['IBES_Timestamp'] = ibes['ANNDATS_ACT'].astype(str)+' '+ibes['ANNTIMS_ACT'].astype(str)
ibes['IBES_Timestamp'] = pd.to_datetime(ibes['IBES_Timestamp'])

In [11]:
ibes['ANNDATS_ACT'] = pd.to_datetime(ibes['ANNDATS_ACT'])

In [12]:
ibes['estdats1'] = np.where(ibes.ANNDATS.dt.weekday == 5,
                            ibes.ANNDATS-bday_us, ibes.ANNDATS)
ibes['estdats1'] = np.where(ibes.ANNDATS.dt.weekday == 6,
                            ibes.ANNDATS-2*bday_us, ibes.estdats1)

In [13]:
# push days on monday if weekend
ibes['repdats1'] = np.where(ibes.ANNDATS_ACT.dt.weekday == 5,
                                 ibes.ANNDATS_ACT+2*bday_us,
                                 ibes.ANNDATS_ACT)
ibes['repdats1'] = np.where(ibes.ANNDATS_ACT.dt.weekday == 6,
                                 ibes.ANNDATS_ACT+bday_us,
                                 ibes.repdats1)

In [14]:
ibes2 = pd.merge(ibes, iclink[['TICKER', 'PERMNO', 'startdate', 'enddate']],
                 on=['TICKER'], how='left')
ibes2['drop'] = np.where((ibes2.IBES_Timestamp < ibes2.startdate) |
                         (ibes2.IBES_Timestamp > ibes2.enddate), 1, 0)
ibes2 = ibes2[ibes2['drop'] != 1]
ibes2.drop('drop', 1, inplace=True)
ibes2 = ibes2[ibes2.FPEDATS < '2020-01-01']
ibes2 = ibes2[ibes2.IBES_Timestamp.notnull()]

# Load the CRSP file
daily stock file for security code 10,11 with exchange code 1, 2, 3

In [15]:
crsp = pd.read_csv('../../Data/CRSP/crsp_dsf_1970_2019_sc10_11_ec123.csv.gz')

In [16]:
adjust = crsp[['date', 'CFACSHR', 'PERMNO']]
adjust['date'] = pd.to_datetime(adjust.date)
adjust['est_factor'] = adjust['CFACSHR']
adjust['rep_factor'] = adjust['CFACSHR']
adjust.drop('CFACSHR', 1, inplace=True)
# keep PERMNO we want
adjust = adjust[adjust.PERMNO.isin(ibes2.PERMNO.unique())]

In [17]:
# if adjustment factors are not the same, adjust the estimate
# to be on the same basis with the actual
ibes2 = pd.merge(ibes2, adjust[['date', 'PERMNO', 'est_factor']], how='left',
                 left_on=['PERMNO', 'estdats1'], right_on=['PERMNO', 'date'])

ibes2 = pd.merge(ibes2, adjust[['date', 'PERMNO', 'rep_factor']], how='left',
                 left_on=['PERMNO', 'repdats1'], right_on=['PERMNO', 'date'])
ibes2.drop(['date_x', 'date_y'], 1, inplace=True)

# adjust the analyst estimates
ibes2['new_value'] = np.where((ibes2.est_factor != ibes2.rep_factor) &
                              (ibes2.est_factor.notnull()) &
                              (ibes2.rep_factor.notnull()),
                              ibes2.rep_factor/ibes2.est_factor*ibes2.VALUE,
                              ibes2.VALUE)

In [18]:
# keep analyst estimates in a 90 day window
ibes2['time_window'] = ibes2.repdats1-ibes2.estdats1
ibes2['time_window'] = ibes2['time_window'].astype('timedelta64[D]')\
                       .astype(int)
ibes2 = ibes2[(ibes2.time_window > 0) & (ibes2.time_window <= 90)]

## Calculate the earning surprises

In [19]:
# Analyst forecasts medians and averages
medest = ibes2.groupby(['TICKER', 'FPEDATS'])[['new_value']].median()\
         .rename(columns={'new_value': 'medest'})
meanest = ibes2.groupby(['TICKER', 'FPEDATS'])[['new_value']].mean()\
         .rename(columns={'new_value': 'meanest'})
numest = ibes2.groupby(['TICKER', 'FPEDATS'])[['new_value']].count()\
         .rename(columns={'new_value': 'numest'})
disp = ibes2.groupby(['TICKER', 'FPEDATS'])[['new_value']].std()\
       .rename(columns={'new_value': 'dispersion'})

surp = pd.concat([medest, meanest, numest, disp], axis=1).reset_index()

# Merge median estimates with ancillary information on permno,
# actuals and report dates.
# Determine whether most analysts are reporting estimates on primary
# or diluted basis following Livnat and Mendenhall (2006)

tmp = ibes2[['TICKER', 'CNAME', 'FPEDATS', 'CUSIP', 'p_count', 'd_count',
             'PERMNO', 'ACTUAL', 'repdats1',
             'ANNDATS_ACT', 'IBES_Timestamp']].drop_duplicates()
surp = pd.merge(surp, tmp, how='left', on=['TICKER', 'FPEDATS'])
surp['basis'] = np.where(surp.p_count <= surp.d_count, 'D', 'P')

# Load Quarterly Compustat

In [20]:
# This compustat is from the COMPUSTAT AND CRSP merged dataset
compu = pd.read_csv('../../Data/Compustat/compustat_qtr_1961_2019.csv.gz')
compu['scaled_net_income'] = compu['niq']/compu['saleq']

compu = compu[(compu.fyr > 0) &
              ((compu.saleq > 0) | (compu.atq > 0)) &
              (compu.consol == 'C') & (compu.popsrc == 'D') &
              (compu.indfmt == 'INDL') & (compu.datafmt == 'STD') &
              (compu.datafqtr.notnull())]

# Calculate the market cap
compu['mcap'] = compu.cshoq*compu.prccq

# Data filtering as in Livnat and Mendenhall (2006)
compu = compu[(compu.rdq.notnull()) & (compu.prccq > 1) &
              (compu.mcap > 5)]

compu.drop(['consol', 'indfmt', 'datafmt', 'popsrc', 'curcdq'], 1, inplace=True)
compu['datadate'] = pd.to_datetime(compu.datadate)

# Create calendar date of fiscal period end in Compustat extract
compu['day'] = 1
compu['mth_adj'] = (-3*(4-compu.fqtr))
compu['date_fyend'] = np.where(compu.fyr <= 5,
                               pd.to_datetime(((compu.fyearq+1)*100+compu.fyr)*100+compu.day, format='%Y%m%d'),
                               pd.to_datetime((compu.fyearq*100+compu.fyr)*100+compu.day, format='%Y%m%d'))
compu['date_fyend'] = compu['date_fyend']+pd.offsets.MonthEnd(0)

# This is slow to run ... I need to think of a faster processing
compu['fqenddt'] = compu.apply(lambda x: x['date_fyend'] +
                               pd.offsets.MonthEnd(x.mth_adj), axis=1)
compu.drop(['day', 'mth_adj'], 1, inplace=True)

# Calculate different actual and expected earnings from COMPUSTAT
compu = compu.sort_values(by=['GVKEY', 'fqtr', 'fyearq'])
for i in ['ajexq', 'epspxq', 'epsfxq', 'cshprq', 'cshfdq', 'spiq', 'scaled_net_income']:
    compu['lag_'+i] = np.where((compu.fyearq-1 == compu.fyearq.shift()) &
                               (compu.fqtr == compu.fqtr.shift()) &
                               (compu.GVKEY == compu.GVKEY.shift()),
                               compu[i].shift(), np.nan)

compu['scaled_net_income_growth'] = (compu['scaled_net_income']-compu['lag_scaled_net_income'])/compu['scaled_net_income']
# Merge GVKEY in the surprise dataframe
cibeslnk = compu[['LPERMNO', 'GVKEY', 'fqenddt']].drop_duplicates()

# Calculate earnings surprises

In [23]:
# assign gvkey in the surp dataframe
surp = pd.merge(surp, cibeslnk, how='left',
                left_on=['PERMNO', 'FPEDATS'], right_on=['LPERMNO', 'fqenddt'])
surp = surp[surp.GVKEY.notnull()]
surp.drop('LPERMNO', 1, inplace=True)

In [24]:
# Calculating the actual and expected surprises as in LM (2006)
# and accounting for exclusions of special items
surp = pd.merge(surp, compu.drop('LPERMNO', 1),
                on=['GVKEY', 'fqenddt'], how='left')

surp['actual1'] = np.where(surp.basis == 'P',
                           surp.epspxq/surp.ajexq,
                           surp.epsfxq/surp.ajexq)
surp['expected1'] = np.where(surp.basis == 'P',
                             surp.lag_epspxq/surp.lag_ajexq,
                             surp.lag_epsfxq/surp.lag_ajexq)

surp['actual2'] = np.where(surp.basis == 'P',
                           (surp.epspxq+0.65*surp.spiq/surp.cshprq)/surp.ajexq,
                           (surp.epsfxq+0.65*surp.spiq/surp.cshfdq)/surp.ajexq)
surp['expected2'] = np.where(surp.basis == 'P',
                             (surp.lag_epspxq+0.65*surp.lag_spiq/surp.lag_cshprq)/surp.lag_ajexq,
                             (surp.lag_epsfxq+0.65*surp.lag_spiq/surp.lag_cshfdq)/surp.lag_ajexq)

surp['sue_rw'] = (surp.actual1-surp.expected1)/(surp.prccq/surp.ajexq)
surp['sue_esi'] = (surp.actual2-surp.expected2)/(surp.prccq/surp.ajexq)
surp['sue_for_med'] = (surp.ACTUAL-surp.medest)/(surp.prccq)
surp['sue_for_mean'] = (surp.ACTUAL-surp.meanest)/(surp.prccq)

In [25]:
# note: to further recalculate these earnigns surprises with forecast
# using stock price 5 days before EA.

# Apply the LM filters and earnings report dates in Compustat
# and in IBES (if available) should not differ by more than one calendar day
surp = surp[(surp.rdq.notnull()) & (surp.prccq > 1) & (surp.mcap > 5)]
surp['rdq'] = pd.to_datetime(surp.rdq)
surp['day_diff'] = surp.rdq-surp.repdats1
surp['day_diff'] = surp['day_diff'].astype('timedelta64[D]')
surp = surp[surp['day_diff'].between(-1, 1)]

surp['ADJ_ACTUAL'] = surp['actual1']
surp['adj_expected_earnings'] = surp['expected1']

surp = surp[['TICKER', 'PERMNO', 'GVKEY', 'CNAME', 'fyr', 'fyearq', 'fqenddt',
             'repdats1', 'ANNDATS_ACT', 'IBES_Timestamp', 'rdq', 'adj_expected_earnings',
             'sue_rw', 'sue_esi', 'sue_for_med', 'sue_for_mean', 'basis', 'ACTUAL', 'ADJ_ACTUAL',
             'medest', 'numest', 'meanest', 'dispersion', 'prccq', 'mcap',
             'ggroup', 'gind', 'gsector', 'gsubind']]


## Get the stock price 5 days before the earnings announcement

In [26]:
#crsp = pd.read_csv('../Data/crsp_2007_2018.csv.gz')
crsp.drop(['EXCHCD', 'SHRCD'], 1, inplace=True)

crsp['date'] = pd.to_datetime(crsp['date'])
crsp['PRC'] = np.abs(crsp['PRC'])

crsp_ = crsp[['date', 'PRC', 'PERMNO']]
crsp_ = crsp_.sort_values(by=['PERMNO', 'date'])
crsp.index = pd.to_datetime(crsp['date'])

crsp_['PRC_s'] = crsp_.groupby('PERMNO')['PRC'].shift(5)  # shift prices

In [27]:
# same procedures as in Hirsleifer paper on drift and attention
surp['ANNDATS_ACT'] = np.where((surp.rdq == surp.ANNDATS_ACT) &
                                 (surp.rdq.dt.year<1990),
                                 surp.ANNDATS_ACT - pd.offsets.Day(1),
                                 surp.ANNDATS_ACT)

In [28]:
# (2) If after 1990, COMPU date is before IBES date, take the compu date
surp['day_diff'] = surp.rdq-surp.ANNDATS_ACT
surp['day_diff'] = surp['day_diff'].astype('timedelta64[D]')

surp['ANNDATS_ACT'] = np.where(surp.day_diff == -1 &
                                 (surp.rdq.dt.year>1990),
                                 surp.rdq, surp.ANNDATS_ACT)
surp.drop('day_diff', 1, inplace=True)

In [29]:
trading_days = pd.DataFrame(index=(np.sort(list(crsp['date'].dt.date.unique()))))

In [30]:
# (3) Create an IBES Adjusted date to get the first trading day following the
# news if the EA is on a holiday or weekend.
repdats1_adj = []
for x in surp['ANNDATS_ACT'].dt.date:
    if x in trading_days:
        repdats1_adj.append(x)
    else:
        repdats1_adj.append(trading_days.loc[x:].index.min())

In [31]:
surp['IBES_Act_Date_Adj'] = pd.to_datetime(repdats1_adj)
# IBES_Act_Date_Adj will be the right day as day 0 for EA days

surp = surp.drop_duplicates()
surp = pd.merge(surp, crsp_, how='left',
                left_on=['PERMNO', 'IBES_Act_Date_Adj'],
                right_on=['PERMNO', 'date'])

In [32]:
# if PRC_s not available in CRSP, because share code != 10 or 11 (e.g., ADR)
# drop these firms
surp = surp[surp.PRC_s.notnull()]

surp['SUE_Med'] = (surp.ACTUAL-surp.medest)/(surp.PRC_s)
surp['SUE_Mean'] = (surp.ACTUAL-surp.meanest)/(surp.PRC_s)

# Similarly to LM (2006) keep obs. where sue_rw is not null
surp = surp[surp.sue_rw.notnull()]
surp = surp[surp.PERMNO.notnull()]

# cases where same PERMNO two different tickers, remove one obs (total 52)
surp['rem'] = np.where((surp.PERMNO == surp.PERMNO.shift()) &
                       (surp.repdats1 == surp.repdats1.shift()) &
                       (surp.TICKER != surp.TICKER.shift()), 1, 0)

# remove possible case of duplicates where duplicate has the exact same info
# but the sue_rw is (erroneously) equals zero
surp['abs_sue'] = np.abs(surp.sue_rw)
surp = surp.sort_values(by=['PERMNO', 'repdats1', 'fqenddt', 'abs_sue'])
surp['rem'] = np.where((surp.PERMNO == surp.PERMNO.shift(-1)) &
                       (surp.repdats1 == surp.repdats1.shift(-1)) &
                       (surp.abs_sue == 0), 1, surp.rem)
surp = surp[surp.rem == 0]

In [33]:
# In very few cases, a rdq in COMPU will have 2 entries
# from two different quarters. Keep the earnings surprise that matches
# the correct quarter and the quarter date in rdq

surp['rem'] = np.where((surp.PERMNO == surp.PERMNO.shift(-1)) &
                       (surp.rdq == surp.rdq.shift(-1)), 1, 0)
surp = surp[surp.rem == 0]

# one case only
surp['rem'] = np.where((surp.IBES_Act_Date_Adj ==
                        surp.IBES_Act_Date_Adj.shift(-1)) &
                       (surp.fqenddt != surp.fqenddt.shift(-1)) &
                       (surp.PERMNO == surp.PERMNO.shift(-1)), 1, 0)  
surp = surp[surp.rem == 0]
surp.drop(['rem', 'abs_sue', 'date'], 1, inplace=True)

# Calculate some measure of price to earnings ratio.
surp['prc_exp_eps'] = surp.PRC_s/surp.medest
surp['prccq_exp_eps'] = surp.prccq/surp.medest

In [34]:
surp['PERMNO'] = surp['PERMNO'].astype(int)
surp['GVKEY'] = surp['GVKEY'].astype(int)

In [35]:
surp.rename(columns={'TICKER':'IBES_TICKER'}, inplace=True)

In [40]:
surp['Year'] = surp['IBES_Timestamp'].dt.year
surp['Quarter'] = surp['IBES_Timestamp'].dt.quarter

In [41]:
surp = surp[['PERMNO', 'IBES_TICKER', 'GVKEY', 'CNAME', 'ANNDATS_ACT', 'IBES_Timestamp', 'IBES_Act_Date_Adj',
             'Year', 'Quarter', 'rdq',
             "SUE_Med", 'sue_rw', 'ACTUAL', 'ADJ_ACTUAL', 'mcap', 'numest', 'dispersion',
             'ggroup', 'gind', 'gsector', 'gsubind', 'medest']]

In [42]:
surp['date'] = pd.to_datetime(surp['IBES_Timestamp'].dt.date)

# Assign S&P index

In [43]:
# merge the SP index constituants
def assignSPIndex(filedir, surp_, index):
    df = pd.read_csv(filedir)
    df = df[['gvkey', 'from', 'thru']].rename(columns={'gvkey':'GVKEY'})
    df['GVKEY'] = df['GVKEY'].astype(int)
    df['date'] = pd.to_datetime(df['from'])
    df['thru'].fillna('2019-12-31', inplace=True)
    df['thru'] = pd.to_datetime(df['thru'])
    df[index] = 1
    df = df.sort_values(by=['date', 'GVKEY'])
    
    surp_ = surp_.sort_values(by=['date', 'GVKEY'])
    surp_ = pd.merge_asof(surp_, df, on='date', by='GVKEY')
    surp_[index] = np.where(surp_.date>surp_['thru'], 0, surp_[index])  # remove firms kicked out of SP500
    surp_ = surp_.sort_values(by=['date', 'PERMNO'])
    surp_ = surp_.drop(['from', 'thru'], 1)
    return surp_

In [44]:
surp = assignSPIndex('../../Data/Compustat/sp500.csv', surp, 'sp500')
surp = assignSPIndex('../../Data/Compustat/sp400.csv', surp, 'sp400')
surp = assignSPIndex('../../Data/Compustat/sp600.csv', surp, 'sp600')
surp = assignSPIndex('../../Data/Compustat/sp1500.csv', surp, 'sp1500')

In [45]:
for i in ['sp500', 'sp400', 'sp600', 'sp1500']:
    surp[i].fillna(0, inplace=True)

In [46]:
surp = surp.drop_duplicates(subset=['PERMNO', 'IBES_Timestamp'], keep='first') # this kills one obs.

# Save output

In [49]:
surp.to_hdf('../../Data/IBES/Earnings_Announcements_and_Surprises.h5', key='panel')